<a href="https://colab.research.google.com/github/CheilaBaiao/GEE_SR/blob/main/NBR_Pantanal_pre_processamento_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#(Prepara o ambiente, autentica no EE e monta o Drive.)
!pip -q install earthengine-api geopandas shapely

from google.colab import drive
drive.mount('/content/drive')

import ee, datetime, json, sys, geopandas as gpd
import shapely.geometry as sgeom
from shapely.geometry import mapping

try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize(project="ee-cheilabaiao")

print("EE OK às", datetime.datetime.now())


Mounted at /content/drive
EE OK às 2025-09-17 03:13:43.038250


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
#(Define anos, resolução/CRS, sentinel NoData e pastas.)
# ===== PARÂMETROS =====
PROJECT        = "ee-cheilabaiao"            # seu projeto EE (opcional no Initialize)
DRIVE_FOLDER   = "Pantanal_TippingPoints_NBR"     # pasta no Google Drive
PREFIX         = "nbr_Pantanal"              # prefixo dos arquivos
START_YEAR     = 1985
END_YEAR       = 2024
CRS            = "EPSG:3857"                  # projeção em METROS
EXPORT_SCALE   = 300                           # 300 m
NODATA         = -32768                        # sentinel seguro para Int16 ×10000

proj300 = ee.Projection(CRS).atScale(EXPORT_SCALE)

print(json.dumps({
    "folder": DRIVE_FOLDER,
    "prefix": PREFIX,
    "years": [START_YEAR, END_YEAR],
    "crs": CRS,
    "scale_m": EXPORT_SCALE,
    "nodata": NODATA
}, indent=2))

{
  "folder": "Pantanal_TippingPoints_NBR",
  "prefix": "nbr_Pantanal",
  "years": [
    1985,
    2024
  ],
  "crs": "EPSG:3857",
  "scale_m": 300,
  "nodata": -32768
}


In [3]:
#(Lê o shapefile, corrige geometrias, dissolve no EE; cria roi_export (bbox) e mask300.)
# ===== SHAPEFILE → pantanal =====
SHP_PATH = '/content/drive/MyDrive/Pantanal/Pantanal.shp'  # <<< ajuste

# Força 2D (sem Z/M) compatível com várias versões do Shapely
def force_2d_any(geom):
    try:
        from shapely import force_2d  # shapely >= 2.0
        return force_2d(geom)
    except Exception:
        if geom.is_empty: return geom
        if isinstance(geom, sgeom.Point): return sgeom.Point(geom.x, geom.y)
        if isinstance(geom, sgeom.LineString): return sgeom.LineString([(x,y) for x,y,*_ in geom.coords])
        if isinstance(geom, sgeom.LinearRing): return sgeom.LinearRing([(x,y) for x,y,*_ in geom.coords])
        if isinstance(geom, sgeom.Polygon):
            ext = [(x,y) for x,y,*_ in geom.exterior.coords]
            ints = [[(x,y) for x,y,*_ in r.coords] for r in geom.interiors]
            return sgeom.Polygon(ext, ints)
        if isinstance(geom, sgeom.MultiPolygon):
            return sgeom.MultiPolygon([force_2d_any(g) for g in geom.geoms])
        if isinstance(geom, sgeom.GeometryCollection):
            return sgeom.GeometryCollection([force_2d_any(g) for g in geom.geoms])
        return geom

gdf = gpd.read_file(SHP_PATH)
if gdf.crs is None or gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)
try:
    gdf = gdf.explode(index_parts=False)
except TypeError:
    gdf = gdf.explode()

gdf['geometry'] = gdf['geometry'].apply(force_2d_any).buffer(0)
gdf = gdf[gdf.is_valid]

# Dissolve server-side (evita RecursionError e alto uso de memória)
features = [ee.Feature(ee.Geometry(mapping(geom), None, False, True)) for geom in gdf.geometry]
pantanal = ee.FeatureCollection(features).geometry()

# Região simples (bbox) em CRS alvo + máscara raster 300 m (1 dentro; fora = máscara)
roi_export = ee.Geometry(pantanal).transform(CRS, 1).bounds(maxError=100)
mask300 = ee.Image.constant(1).toByte().reproject(proj300).clip(pantanal)

# Diagnóstico rápido de área (pixelArea na mesma grade)
area_m2 = (ee.Image.pixelArea().reproject(proj300).updateMask(mask300)
           .reduceRegion(ee.Reducer.sum(), geometry=roi_export, crs=CRS,
                         scale=EXPORT_SCALE, bestEffort=True, tileScale=16, maxPixels=1e13)
           .get('area').getInfo())
print('Área (km²) ~', area_m2/1e6)


Área (km²) ~ 151806.60001063062


In [4]:
# 4) (Converte SR para reflectância, define SWIR/NIR por sensor e calcula nbr com normalizedDifference.)
# ===== FUNÇÕES LANDSAT L2 =====
def scale_sr(img):
    sr = img.select('SR_B.*').multiply(0.0000275).add(-0.2)   # SR→reflectância
    return img.addBands(sr, overwrite=True)

def add_swir_nir(img, sensor):
    if sensor == 'L89':   # Landsat 8/9
        swir = img.select('SR_B7'); nir = img.select('SR_B5')
    else:                 # Landsat 5/7
        swir = img.select('SR_B7'); nir = img.select('SR_B4')
    return img.addBands([swir.rename('SWIR'), nir.rename('NIR')])

def add_nbr(img):
    return img.addBands(img.normalizedDifference(['NIR','SWIR']).rename('nbr'))

# ===== COLEÇÕES nbr =====
L5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
      .filterBounds(pantanal).filterDate('1985-01-01', '2013-07-01')
      .map(scale_sr).map(lambda im: add_swir_nir(im,'L57')).map(add_nbr)
      .select(['nbr']))

L7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
      .filterBounds(pantanal).filterDate('1999-01-01', '2024-12-31')
      .map(scale_sr).map(lambda im: add_swir_nir(im,'L57')).map(add_nbr)
      .select(['nbr']))

L8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
      .filterBounds(pantanal).filterDate('2013-04-11', '2024-12-31')
      .map(scale_sr).map(lambda im: add_swir_nir(im,'L89')).map(add_nbr)
      .select(['nbr']))

L9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
      .filterBounds(pantanal).filterDate('2021-11-01', '2024-12-31')
      .map(scale_sr).map(lambda im: add_swir_nir(im,'L89')).map(add_nbr)
      .select(['nbr']))

LS_nbr = L5.merge(L7).merge(L8).merge(L9)
print('Cenas nbr na coleção total:', LS_nbr.size().getInfo())


Cenas nbr na coleção total: 17618


In [5]:
# 5) (Por cena: 30→300 m (média) dentro da bbox; depois mediana temporal a 300 m; aplica mask300 e sentinel.)
# ===== nbr mensal (300 m) e stack anual =====
def monthly_nbr_300m(year: int, month: int) -> ee.Image:
    ini = ee.Date.fromYMD(year, month, 1); end = ini.advance(1, 'month')
    coll = LS_nbr.filterDate(ini, end).filterBounds(roi_export)
    nscenes = coll.size()

    # Por cena: recorta à bbox e agrega 30 m → 300 m (média); define grade fixa
    def _to300(im):
        return (im.select('nbr')
                  .clip(roi_export)
                  .reduceResolution(ee.Reducer.mean(), maxPixels=65535)
                  .setDefaultProjection(proj300))

    coll300 = coll.map(_to300)

    # Mediana temporal já em 300 m; aplica máscara raster do Pantanal
    nd300 = ee.Image(ee.Algorithms.If(nscenes.gt(0), coll300.median(), ee.Image(0).updateMask(ee.Image(0))))
    nd300 = nd300.updateMask(mask300)

    im = (nd300.multiply(10000).round().toInt16()
               .unmask(NODATA)
               .rename(f'nbr_{month:02d}')
               .set({'system:time_start': ini.millis(), 'count': nscenes}))
    return im

def yearly_stack_300m(year: int) -> ee.Image:
    bands = [monthly_nbr_300m(year, m) for m in range(1,13)]
    return (ee.Image.cat(bands)
            .clip(roi_export)  # bbox simples (barato)
            .set({'year': year, 'system:time_start': ee.Date.fromYMD(year,1,1).millis()}))


In [6]:
# (Usa a mesma grade/região do export; ignora sentinel na estatística.)
# ===== QA pré-export: percentis mensais (mesma grade/região do export) =====
import pandas as pd

def monthly_percentiles_300m(year: int, perc=[5,25,50,75,95]) -> pd.DataFrame:
    rows=[]
    for m in range(1,13):
        im = monthly_nbr_300m(year, m)
        im_valid = im.updateMask(im.neq(NODATA))
        s = (im_valid.reduceRegion(
                reducer=ee.Reducer.percentile(perc),
                geometry=roi_export, crs=CRS, scale=EXPORT_SCALE,
                bestEffort=True, tileScale=16, maxPixels=1e13)
             .getInfo() or {})
        row={'band': f'nbr_{m:02d}'}
        for p in perc:
            key=f'nbr_{m:02d}_p{p}'
            val = s.get(key)
            row[f'p{p}'] = (val/10000.0) if val is not None else None
        rows.append(row)
    return pd.DataFrame(rows)

TEST_YEAR = 1985
df_pre = monthly_percentiles_300m(TEST_YEAR)
df_pre


,band,p5,p25,p50,p75,p95
0,nbr_01,0.188768,0.355242,0.438351,0.559973,0.687928
1,nbr_02,0.393685,0.496000,0.566375,0.630336,0.713491
2,nbr_03,0.297569,0.470356,0.566379,0.655939,0.739061
3,nbr_04,0.377734,0.531255,0.620804,0.684683,0.761150
4,nbr_05,0.313690,0.428749,0.531298,0.646350,0.748455
5,nbr_06,0.214413,0.342381,0.451143,0.566345,0.694288
6,nbr_07,0.112006,0.246369,0.355142,0.483131,0.668740
7,nbr_08,0.092821,0.227150,0.361532,0.534345,0.745538
8,nbr_09,0.006494,0.134380,0.249487,0.415944,0.633422
9,nbr_10,0.019276,0.198387,0.351929,0.531168,0.697358


In [7]:
# 7) (Exporta um ano como stack de 12 bandas, com noData e bbox simples; inclui monitor simples das tasks.)
# ===== Export anual @ 300 m (COG) =====
def export_one_year(year: int):
    img = yearly_stack_300m(year)
    desc = f'{PREFIX}_{year}_stack_300m'
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=desc,
        folder=DRIVE_FOLDER,
        fileNamePrefix=desc,
        region=roi_export,              # bbox simples
        crs=CRS,
        scale=EXPORT_SCALE,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True, 'noData': NODATA}
    )
    task.start()
    print('> Export iniciado:', desc)

# Exemplo: exporta o ano de teste
export_one_year(TEST_YEAR)

# Monitor
for t in ee.batch.Task.list():
    st = t.status()
    print('-', st.get('description'), '|', st.get('state'))


> Export iniciado: nbr_Pantanal_1985_stack_300m
- nbr_Pantanal_1985_stack_300m | READY
- NDVI_Pantanal_2000_stack_300m | COMPLETED
- NDVI_Pantanal_1999_stack_300m | COMPLETED
- NDVI_Pantanal_1998_stack_300m | COMPLETED
- NDVI_Pantanal_1997_stack_300m | COMPLETED
- NDVI_Pantanal_1996_stack_300m | COMPLETED
- NDVI_Pantanal_1995_stack_300m | COMPLETED
- NDVI_Pantanal_1994_stack_300m | COMPLETED
- NDVI_Pantanal_1993_stack_300m | COMPLETED
- NDVI_Pantanal_1992_stack_300m | COMPLETED
- NDVI_Pantanal_1991_stack_300m | COMPLETED
- NDVI_Pantanal_1990_stack_300m | COMPLETED
- NDVI_Pantanal_1989_stack_300m | COMPLETED
- NDVI_Pantanal_1988_stack_300m | COMPLETED
- NDVI_Pantanal_1987_stack_300m | COMPLETED
- NDVI_Pantanal_1986_stack_300m | COMPLETED
- NDVI_Pantanal_1985_stack_300m | COMPLETED
- NDVI_Pantanal_2004_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | CANCELLED
- NDVI_Pantanal_2005_stack_300m |

In [8]:
# 8) (Conta cenas por mês em um ano para entender vazios.)

def months_check(y):
    empty=[]; counts=[]
    for m in range(1,13):
        c = (LS_nbr.filterBounds(roi_export)
                     .filterDate(ee.Date.fromYMD(y,m,1), ee.Date.fromYMD(y,m,1).advance(1,'month'))
                     .size().getInfo())
        counts.append(int(c))
        if c==0: empty.append(m)
    print(f'{y}: cenas/mês =', counts, '| meses vazios:', empty or 'nenhum')

months_check(TEST_YEAR)


1985: cenas/mês = [9, 14, 14, 9, 13, 13, 18, 17, 17, 16, 13, 13] | meses vazios: nenhum


In [9]:
# 9) (Confere dtype/NoData/COG e percentis; devem bater com a célula 6, salvo diferenças numéricas pequenas.)
!pip -q install rasterio numpy

import rasterio, numpy as np, os, math, pandas as pd

tif_path = "/content/drive/MyDrive/Pantanal_TippingPoints_NBR/nbr_Pantanal_1985_stack_300m.tif"

with rasterio.open(tif_path) as ds:
    print("Arquivo:", os.path.basename(tif_path))
    print("Bandas:", ds.count)
    print("CRS   :", ds.crs)
    print("Dtype :", ds.dtypes[0])
    print("NoData:", ds.nodatavals[0])
    print("Tiled :", ds.profile.get("tiled", None))
    print("Overviews b1:", ds.overviews(1))
    print("Transform:", ds.transform)
    if ds.crs and ds.crs.is_projected:
        print(f"Pixel ~ {abs(ds.transform.a):.2f} m × {abs(ds.transform.e):.2f} m")

    scale = 10000.0 if ds.dtypes[0]=='int16' else 1.0
    H, W = ds.height, ds.width
    stats=[]
    for b in range(1, ds.count+1):
        arr = ds.read(b, masked=True).astype('float32')
        vals = (arr.compressed()/scale) if arr.mask is not None else (arr/scale).ravel()
        if vals.size==0:
            stats.append({'band': f'nbr_{b:02d}','p5':None,'p25':None,'p50':None,'p75':None,'p95':None})
        else:
            stats.append({
                'band': f'nbr_{b:02d}',
                'p5': float(np.percentile(vals,5)),
                'p25': float(np.percentile(vals,25)),
                'p50': float(np.percentile(vals,50)),
                'p75': float(np.percentile(vals,75)),
                'p95': float(np.percentile(vals,95)),
            })

df_post = pd.DataFrame(stats)
display(df_post)
print("\nDiferenças |pós - pré| (esperado ~0):")
df_cmp = df_post.merge(df_pre, on='band', suffixes=('_post','_pre'))
for p in [5,25,50,75,95]:
    df_cmp[f'd_p{p}'] = (df_cmp[f'p{p}_post'] - df_cmp[f'p{p}_pre']).abs()
display(df_cmp[[f'd_p{p}' for p in [5,25,50,75,95]]].round(4))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 82.5 MB/s eta 0:00:00
Arquivo: nbr_Pantanal_1985_stack_300m.tif
Bandas: 12
CRS   : EPSG:3857
Dtype : int16
NoData: -32768.0
Tiled : True
Overviews b1: [2, 4, 8, 16, 32]
Transform: | 300.00, 0.00,-6583800.00|
| 0.00,-300.00,-1763700.00|
| 0.00, 0.00, 1.00|
Pixel ~ 300.00 m × 300.00 m


,band,p5,p25,p50,p75,p95
0,nbr_01,0.1863,0.35270,0.4356,0.5585,0.6863
1,nbr_02,0.3910,0.49600,0.5676,0.6335,0.7149
2,nbr_03,0.2981,0.46890,0.5691,0.6574,0.7392
3,nbr_04,0.3730,0.53565,0.6246,0.6896,0.7614
4,nbr_05,0.3123,0.43110,0.5339,0.6430,0.7437
5,nbr_06,0.2143,0.34210,0.4495,0.5647,0.6924
6,nbr_07,0.1144,0.24520,0.3526,0.4830,0.6669
7,nbr_08,0.0906,0.22560,0.3610,0.5354,0.7447
8,nbr_09,0.0098,0.13810,0.2511,0.4136,0.6399
9,nbr_10,0.0163,0.19500,0.3523,0.5284,0.6926



Diferenças |pós - pré| (esperado ~0):


,d_p5,d_p25,d_p50,d_p75,d_p95
0,0.0025,0.0025,0.0028,0.0015,0.0016
1,0.0027,0.0000,0.0012,0.0032,0.0014
2,0.0005,0.0015,0.0027,0.0015,0.0001
3,0.0047,0.0044,0.0038,0.0049,0.0002
4,0.0014,0.0024,0.0026,0.0033,0.0048
5,0.0001,0.0003,0.0016,0.0016,0.0019
6,0.0024,0.0012,0.0025,0.0001,0.0018
7,0.0022,0.0015,0.0005,0.0011,0.0008
8,0.0033,0.0037,0.0016,0.0023,0.0065
9,0.0030,0.0034,0.0004,0.0028,0.0048
